In [72]:
import sys
import os
from scipy import stats
import pandas as pd
import numpy as np
import seaborn as sns
thisDir = os.path.expanduser('~/Desktop/MSC_Alexis/analysis/')
dataDir = thisDir + 'data/mvpa_data/'
outDir = thisDir + 'output/results/acc/'
ALL=pd.read_csv(outDir+'/ALL/acc.csv')
#SS
SS=pd.read_csv(outDir+'SS/acc.csv')
BS=pd.read_csv(outDir+'BS/acc.csv')
#DS
DS=pd.read_csv(outDir+'DS/acc.csv')
#CV
CV=pd.read_csv(outDir+'CV/acc.csv')

#motor train, mem test
SS_motorMem=SS[(SS['train_task']=='motor') & (SS['test_task']=='mem')].acc.values
BS_motorMem=BS[(BS['train_task']=='motor') & (BS['test_task']=='mem')].acc.values
#motor train mix test
SS_motorSem=SS[(SS['train_task']=='motor') & (SS['test_task']=='semantic')].acc.values
BS_motorSem=BS[(BS['train_task']=='motor') & (BS['test_task']=='semantic')].acc.values
#motor train mix test
SS_motorGlass=SS[(SS['train_task']=='motor') & (SS['test_task']=='glass')].acc.values
BS_motorGlass=BS[(BS['train_task']=='motor') & (BS['test_task']=='glass')].acc.values

#sem train, mem test
SS_semMem=SS[(SS['train_task']=='semantic') & (SS['test_task']=='mem')].acc.values
BS_semMem=BS[(BS['train_task']=='semantic') & (BS['test_task']=='mem')].acc.values
#sem train motor test
SS_semMotor=SS[(SS['train_task']=='semantic') & (SS['test_task']=='motor')].acc.values
BS_semMotor=BS[(BS['train_task']=='semantic') & (BS['test_task']=='motor')].acc.values
#sem train glass test
SS_semGlass=SS[(SS['train_task']=='semantic') & (SS['test_task']=='glass')].acc.values
BS_semGlass=BS[(BS['train_task']=='semantic') & (BS['test_task']=='glass')].acc.values

#glass train mem test
SS_glassMem=SS[(SS['train_task']=='glass') & (SS['test_task']=='mem')].acc.values
BS_glassMem=BS[(BS['train_task']=='glass') & (BS['test_task']=='mem')].acc.values
#glass train motor test
SS_glassMotor=SS[(SS['train_task']=='glass') & (SS['test_task']=='motor')].acc.values
BS_glassMotor=BS[(BS['train_task']=='glass') & (BS['test_task']=='motor')].acc.values
#glass train sem test
SS_glassSem=SS[(SS['train_task']=='glass') & (SS['test_task']=='semantic')].acc.values
BS_glassSem=BS[(BS['train_task']=='glass') & (BS['test_task']=='semantic')].acc.values

#memory train, mix test
SS_memSem=SS[(SS['train_task']=='mem') & (SS['test_task']=='semantic')].acc.values
BS_memSem=BS[(BS['train_task']=='mem') & (BS['test_task']=='semantic')].acc.values
#memory train, mix test
SS_memGlass=SS[(SS['train_task']=='mem') & (SS['test_task']=='glass')].acc.values
BS_memGlass=BS[(BS['train_task']=='mem') & (BS['test_task']=='glass')].acc.values
#memory train motor test
SS_memMotor=SS[(SS['train_task']=='mem') & (SS['test_task']=='motor')].acc.values
BS_memMotor=BS[(BS['train_task']=='mem') & (BS['test_task']=='motor')].acc.values


#motor
DS_motor=DS[DS.task=='motor'].acc.values
CV_motor=CV[CV.task=='motor'].acc.values

#Glass
DS_glass=DS[DS.task=='glass'].acc.values
CV_glass=CV[CV.task=='glass'].acc.values

#Semantic
DS_sem=DS[DS.task=='semantic'].acc.values
CV_sem=CV[CV.task=='semantic'].acc.values

#memory
DS_mem=DS[DS.task=='mem'].acc.values
CV_mem=CV[CV.task=='mem'].acc.values



In [32]:
#t-test CV DS
mot=stats.ttest_ind(CV_motor,DS_motor, equal_var = False)
sem=stats.ttest_ind(CV_sem, DS_sem, equal_var = False)
glas=stats.ttest_ind(CV_glass,DS_glass,  equal_var = False)
mem=stats.ttest_ind(CV_mem, DS_mem, equal_var = False)
ST_ttest=pd.DataFrame()
ST_ttest['task']=['mot','sem','glas','mem']
ST_ttest['t_val']=mot[0],sem[0],glas[0],mem[0]
ST_ttest['p_val']=mot[1],sem[1],glas[1],mem[1]

In [33]:
ST_ttest


,task,t_val,p_val
0,mot,8.529634,5.818171e-12
1,sem,12.207735,7.788849e-17
2,glas,11.400504,4.825065e-08
3,mem,6.709941,8.088258e-09


In [38]:
#t-test SS BS different task analysis
motMem=stats.ttest_ind(SS_motorMem, BS_motorMem, equal_var = False)
motSem=stats.ttest_ind(SS_motorSem, BS_motorSem, equal_var = False)
motGla=stats.ttest_ind(SS_motorGlass, BS_motorGlass, equal_var = False)
semMem=stats.ttest_ind(SS_semMem, BS_semMem, equal_var = False)
semMot=stats.ttest_ind(SS_semMotor, BS_semMotor, equal_var = False)
semGla=stats.ttest_ind(SS_semGlass, BS_semGlass, equal_var = False)
memSem=stats.ttest_ind(SS_memSem, BS_memSem, equal_var = False)
memGla=stats.ttest_ind(SS_memGlass, BS_memGlass, equal_var = False)
memMot=stats.ttest_ind(SS_memMotor, BS_memMotor, equal_var = False)
glaSem=stats.ttest_ind(SS_glassSem, BS_glassSem, equal_var = False)
glaMem=stats.ttest_ind(SS_glassMem, BS_glassMem, equal_var = False)
glaMot=stats.ttest_ind(SS_glassMotor, BS_glassMotor, equal_var = False)

DT_ttest=pd.DataFrame()
DT_ttest['train_task']=['motor','motor','motor','semantic','semantic','semantic','memory','memory','memory','glass','glass','glass']
DT_ttest['test_task']=['memory','semantic','glass','memory','motor','glass','semantic','glass','motor','semantic','memory','motor']
DT_ttest['t_val']=motMem[0],motSem[0],motGla[0],semMem[0],semMot[0],semGla[0],memSem[0],memGla[0],memMot[0],glaSem[0],glaMem[0],glaMot[0]
DT_ttest['p_val']=motMem[1],motSem[1],motGla[1],semMem[1],semMot[1],semGla[1],memSem[1],memGla[1],memMot[1],glaSem[1],glaMem[1],glaMot[1]



In [39]:
#SS BS
DT_ttest

,train_task,test_task,t_val,p_val
0,motor,memory,0.309434,7.635982e-01
1,motor,semantic,0.151635,8.829130e-01
2,motor,glass,0.985825,3.534219e-01
3,semantic,memory,2.901866,1.921674e-02
4,semantic,motor,2.165621,5.929566e-02
5,semantic,glass,6.696369,1.133234e-04
6,memory,semantic,2.924317,1.219066e-02
7,memory,glass,1.454935,1.803402e-01
8,memory,motor,1.206284,2.580200e-01
9,glass,semantic,11.121875,3.680379e-07


In [40]:
#t-test CV SS
motMem=stats.ttest_ind(CV_motor, SS_motorMem, equal_var = False)
motSem=stats.ttest_ind(CV_motor,SS_motorSem, equal_var = False)
motGla=stats.ttest_ind(CV_motor, SS_motorGlass,equal_var = False)
semMem=stats.ttest_ind(CV_sem, SS_semMem,equal_var = False)
semMot=stats.ttest_ind(CV_sem, SS_semMotor,equal_var = False)
semGla=stats.ttest_ind(CV_sem, SS_semGlass, equal_var = False)
memSem=stats.ttest_ind(CV_mem, SS_memSem, equal_var = False)
memGla=stats.ttest_ind(CV_mem,SS_memGlass,  equal_var = False)
memMot=stats.ttest_ind(CV_mem, SS_memMotor, equal_var = False)
glaSem=stats.ttest_ind(CV_glass,SS_glassSem, equal_var = False)
glaMem=stats.ttest_ind(CV_glass, SS_glassMem, equal_var = False)
glaMot=stats.ttest_ind(CV_glass, SS_glassMotor,equal_var = False)

CV_SS_ttest=pd.DataFrame()
CV_SS_ttest['train_task']=['motor','motor','motor','semantic','semantic','semantic','memory','memory','memory','glass','glass','glass']
CV_SS_ttest['test_task']=['memory','semantic','glass','memory','motor','glass','semantic','glass','motor','semantic','memory','motor']
CV_SS_ttest['t_val']=motMem[0],motSem[0],motGla[0],semMem[0],semMot[0],semGla[0],memSem[0],memGla[0],memMot[0],glaSem[0],glaMem[0],glaMot[0]
CV_SS_ttest['p_val']=motMem[1],motSem[1],motGla[1],semMem[1],semMot[1],semGla[1],memSem[1],memGla[1],memMot[1],glaSem[1],glaMem[1],glaMot[1]


In [41]:
CV_SS_ttest

,train_task,test_task,t_val,p_val
0,motor,memory,9.207656,0.000018
1,motor,semantic,7.770788,0.000072
2,motor,glass,6.061427,0.000405
3,semantic,memory,5.530476,0.000568
4,semantic,motor,9.021395,0.000011
5,semantic,glass,3.639889,0.006081
6,memory,semantic,4.116504,0.003578
7,memory,glass,3.024009,0.018575
8,memory,motor,2.718405,0.028667
9,glass,semantic,-0.624943,0.542257


In [42]:
#t-test DS BS
motMem=stats.ttest_ind(DS_motor, BS_motorMem, equal_var = False)
motSem=stats.ttest_ind(DS_motor, BS_motorSem, equal_var = False)
motGla=stats.ttest_ind(DS_motor, BS_motorGlass, equal_var = False)
semMem=stats.ttest_ind(DS_sem, BS_semMem, equal_var = False)
semMot=stats.ttest_ind(DS_sem, BS_semMotor, equal_var = False)
semGla=stats.ttest_ind(DS_sem, BS_semGlass, equal_var = False)
memSem=stats.ttest_ind(DS_mem, BS_memSem, equal_var = False)
memGla=stats.ttest_ind(DS_mem, BS_memGlass, equal_var = False)
memMot=stats.ttest_ind(DS_mem, BS_memMotor, equal_var = False)
glaSem=stats.ttest_ind(DS_glass, BS_glassSem, equal_var = False)
glaMem=stats.ttest_ind(DS_glass, BS_glassMem, equal_var = False)
glaMot=stats.ttest_ind(DS_glass, BS_glassMotor, equal_var = False)

DS_BS_ttest=pd.DataFrame()
DS_BS_ttest['train_task']=['motor','motor','motor','semantic','semantic','semantic','memory','memory','memory','glass','glass','glass']
DS_BS_ttest['test_task']=['memory','semantic','glass','memory','motor','glass','semantic','glass','motor','semantic','memory','motor']
DS_BS_ttest['t_val']=motMem[0],motSem[0],motGla[0],semMem[0],semMot[0],semGla[0],memSem[0],memGla[0],memMot[0],glaSem[0],glaMem[0],glaMot[0]
DS_BS_ttest['p_val']=motMem[1],motSem[1],motGla[1],semMem[1],semMot[1],semGla[1],memSem[1],memGla[1],memMot[1],glaSem[1],glaMem[1],glaMot[1]


In [43]:
DS_BS_ttest

,train_task,test_task,t_val,p_val
0,motor,memory,6.536882,2.567688e-09
1,motor,semantic,6.642845,1.523164e-09
2,motor,glass,7.463042,4.353196e-11
3,semantic,memory,4.634606,1.134731e-05
4,semantic,motor,5.194097,1.283781e-06
5,semantic,glass,5.086131,1.975503e-06
6,memory,semantic,5.000056,2.248209e-06
7,memory,glass,5.864997,5.474335e-08
8,memory,motor,3.481101,7.256838e-04
9,glass,semantic,-0.019307,9.846311e-01


In [76]:
ALL.std()

cv_acc    0.010809
acc       0.114354
dtype: float64

In [77]:
stats.ttest_ind(ALL.cv_acc.values, ALL.acc.values, equal_var = False)

Ttest_indResult(statistic=19.03793849097413, pvalue=4.015709941989158e-26)

In [38]:
#Difference score CVmDS permutation
pST=pd.read_csv(thisDir+'output/results/permutation/CVmDS_acc.csv',na_values=[9999])


DS=pd.read_csv(outDir+'DS/acc.csv')
#CV
CV=pd.read_csv(outDir+'CV/acc.csv')
#permutation testing CV DS
DS.rename(columns={'train_sub':'sub'},inplace=True)
same_task=CV.merge(DS,how='left',on=['task','sub'],suffixes=('','_DS'))
#SS-OS
same_task['diff'] =same_task.acc-same_task.acc_DS 
#subset so we only average the SS-OS per train/test tasks
ST=same_task[['task','diff']]
#take average
diffST=ST.groupby(['task']).mean()
diffST.reset_index(inplace=True)

#prepping BS and SS
BS.rename(columns={'train_sub':'sub'},inplace=True)

diff_task=SS.merge(BS,how='left',on=['train_task','test_task','sub'],suffixes=('','_BS'))
#SS-OS
diff_task['diff'] =diff_task.acc-diff_task.acc_BS 
#subset so we only average the SS-OS per train/test tasks
DT=diff_task[['train_task','test_task','diff']]
#take average
diffDT=DT.groupby(['train_task','test_task']).mean()
diffDT.reset_index(inplace=True)
diffDT['TrTe'] = diffDT.train_task.map(str) + "_" + diffDT.test_task
DT=diffDT[['TrTe','diff']]
DT.set_index('TrTe',inplace=True)

#permuted different task (SS-OS) SS-BS
pDT=pd.read_csv(thisDir+'output/results/permutation/SSmBS_acc.csv',na_values=[9999])


#dataframe of pvals for each task in same task 
#diffST.set_index('task',inplace=True)
tasks=[]
p_val=[]
taskList=['mem','motor','semantic','glass']
for task in taskList:
    muST=diffST.loc[diffST['task']==task]
    muST=muST.iloc[0,1]
    ST_perm=pST.loc[pST['task']==task]
    ST_perm_keep=ST_perm[ST_perm['diff']>muST]
    count=ST_perm_keep['diff'].count()
    p=(count+1)/1001
    tasks.append(task)
    p_val.append(p)
ST_all=pd.DataFrame({'task':tasks, 'pval':p_val})
ST_all.set_index('task',inplace=True)
ST_all

,pval
task,
mem,0.000999
motor,0.000999
semantic,0.000999
glass,0.000999


In [39]:
#permutation testing SS BS
#dataframe to hold p values for each task by task
training=[]
testing=[]
p_values=[]
for train_task in taskList:
    for test_task in taskList:
        if train_task==test_task:
            continue
        else:
            DTmu=DT.loc[train_task+'_'+test_task].values
            DT_perm=pDT.loc[(pDT['train_task']==train_task) & (pDT['test_task']==test_task)]
            DT_perm_kept=DT_perm[DT_perm['diff']>DTmu[0]]
            count=DT_perm_kept['diff'].count()
            pval=(count+1)/(1001)
            training.append(train_task)
            testing.append(test_task)
            p_values.append(pval)
DT_all=pd.DataFrame({'train_task':training,'test_task':testing,'pval':p_values})
DT_all

,train_task,test_task,pval
0,mem,motor,0.129870
1,mem,semantic,0.018981
2,mem,glass,0.022977
3,motor,mem,0.079920
4,motor,semantic,0.197802
5,motor,glass,0.003996
6,semantic,mem,0.000999
7,semantic,motor,0.002997
8,semantic,glass,0.000999
9,glass,mem,0.000999


In [69]:
#permutation testing CV SS
###############################
#Will need to change directory#
###############################
#permuted labels SS
SS_permuted=pd.read_csv(thisDir+'output/results/permutation/SS_acc.csv',na_values=[9999])

#actual scores SS 
SS=pd.read_csv(outDir+'SS/acc.csv')


#subset so we only average train/test tasks
SS=SS[['train_task','test_task','acc']]
#take average
avg=SS.groupby(['train_task', 'test_task']).mean()
avg.reset_index(inplace=True)
avg['TrTe'] = avg.train_task.map(str) + "_" + avg.test_task
avg=avg[['TrTe','acc']]
avg.set_index('TrTe',inplace=True)



In [70]:

#dataframe to hold p values for each task by task
training=[]
testing=[]
p_values=[]
for train_task in taskList:
    for test_task in taskList:
        if train_task==test_task:
            continue
        else:
            Scmu=avg.loc[train_task+'_'+test_task].values
            btw=SS_permuted.loc[(SS_permuted['train_task']==train_task) & (SS['test_task']==test_task)]
            btw_kept=btw[btw['diff']>Scmu[0]]
            count=btw_kept['diff'].count()
            pval=(count+1)/(1001)
            training.append(train_task)
            testing.append(test_task)
            p_values.append(pval)
btw_all=pd.DataFrame({'train_task':training,'test_task':testing,'pval':p_values})
btw_all

,train_task,test_task,pval
0,mem,motor,0.000999
1,mem,semantic,0.000999
2,mem,glass,0.000999
3,motor,mem,0.000999
4,motor,semantic,0.000999
5,motor,glass,0.000999
6,semantic,mem,0.000999
7,semantic,motor,0.000999
8,semantic,glass,0.000999
9,glass,mem,0.000999


In [60]:
between_permuted


,train_task,test_task,diff
0,glass,mem,0.021458
1,glass,motor,0.022639
2,glass,semantic,0.028958
3,mem,glass,0.063472
4,mem,motor,-0.086875
...,...,...,...
11995,motor,mem,0.057500
11996,motor,semantic,0.041875
11997,semantic,glass,0.099306
11998,semantic,mem,-0.036875


In [53]:
SS.head()

,train_task,test_task,sub,acc,spec,sen
0,glass,semantic,MSC01,0.950,1.0,0.90
1,glass,motor,MSC01,0.945,1.0,0.89
2,glass,mem,MSC01,0.830,1.0,0.66
3,semantic,glass,MSC01,0.850,1.0,0.70
4,semantic,motor,MSC01,0.755,1.0,0.51


In [47]:
diffST

,train_task,test_task,diff
0,glass,mem,0.048333
1,glass,motor,0.069861
2,glass,semantic,-0.057153
3,mem,glass,0.115069
4,mem,motor,0.105625
5,mem,semantic,0.076875
6,motor,glass,0.242847
7,motor,mem,0.287500
8,motor,semantic,0.318125
9,semantic,glass,0.099167


In [72]:
#Calculating SD and Mean for all regular analysis
def statsACC(df,analysis):
    if analysis=='CV':
        print('cross validation stats')
        stats=df.groupby('task').mean()
        stats.rename(columns={'acc':'Mean'},inplace=True)
        sd=df.groupby('task').std()
        stats['Std']=sd['acc']
        stats.to_csv(outDir+analysis+'/stats.csv', index=True)
    elif analysis=='SS':
        print('same sub stats')
        df.drop(['sub'], axis=1, inplace=True)
        stats=df.groupby(['test_task', 'train_task']).mean()
        stats.rename(columns={'acc':'Mean'}, inplace=True)
        sd=df.groupby(['test_task', 'train_task']).std()
        stats['Std']=sd['acc']
        stats.reset_index(inplace=True)
        stats.to_csv(outDir+analysis+'/stats.csv', index=True)
    elif analysis=='DS':
        print('diff sub stats')
        df.drop(['train_sub', 'test_sub'], axis=1, inplace=True)
        stats=df.groupby('task').mean()
        stats.rename(columns={'acc':'Mean'}, inplace=True)
        sd=df.groupby('task').std()
        stats['Std']=sd['acc']
        stats.to_csv(outDir+analysis+'/stats.csv', index=True)
    elif analysis=='BS':
        print('diff sub diff task stats')
        df.drop(['train_sub', 'test_sub'], axis=1, inplace=True)
        stats=df.groupby(['test_task', 'train_task']).mean()
        stats.rename(columns={'acc':'Mean'}, inplace=True)
        sd=df.groupby(['test_task', 'train_task']).std()
        stats['Std']=sd['acc']
        stats.reset_index(inplace=True)
        stats.to_csv(outDir+analysis+'/stats.csv', index=True)
    return stats

In [79]:
#statsACC(CV,'CV')
statsACC(SS,'SS')
#statsACC(DS,'DS')
#statsACC(BS,'BS')txt = "apple, banana, cherry"

# setting the maxsplit parameter to 1, will return a list with 2 elements!
x = txt.rsplit(", ", 1)

print(x)



same sub stats


,test_task,train_task,Mean,Std
0,glass,mem,0.841806,0.141014
1,glass,motor,0.658750,0.148246
2,glass,semantic,0.807083,0.101249
3,mem,glass,0.760694,0.139531
4,mem,motor,0.632500,0.103957
5,mem,semantic,0.694375,0.123446
6,motor,glass,0.792569,0.142429
7,motor,mem,0.876250,0.120971
8,motor,semantic,0.628750,0.093417
9,semantic,glass,0.900069,0.073824


In [82]:
ALL['cv_acc'].mean()
ALL['cv_acc'].std()

0.010808903106404302

In [84]:
#ALL['acc'].mean()
ALL['acc'].std()

0.11435408419187446

In [26]:
SS=pd.read_csv(outDir+'SS/acc.csv')

In [30]:
SS.groupby(['train_task']).mean()

,acc,spec,sen
train_task,,,
glass,0.858125,0.986250,0.730000
mem,0.894560,0.987731,0.801389
motor,0.698426,0.999583,0.397269
semantic,0.773403,0.999583,0.547222


In [22]:
SS.groupby(['train_task','test_task']).mean()

acc      spec       sen
train_task test_task                              
glass      mem        0.830139  0.986250  0.674028
           motor      0.808611  0.973750  0.643472
           semantic   0.935625  0.998750  0.872500
mem        glass      0.878681  0.984444  0.772917
           motor      0.888125  0.983750  0.792500
           semantic   0.916875  0.995000  0.838750
motor      glass      0.738403  1.000000  0.476806
           mem        0.693750  0.998750  0.388750
           semantic   0.663125  1.000000  0.326250
semantic   glass      0.844583  1.000000  0.689167
           mem        0.772500  0.998750  0.546250
           motor      0.703125  1.000000  0.406250

In [23]:
SS.groupby(['train_task','test_task']).std()

acc      spec       sen
train_task test_task                              
glass      mem        0.114660  0.038891  0.206827
           motor      0.153236  0.066534  0.262088
           semantic   0.051508  0.003536  0.102365
mem        glass      0.108183  0.016211  0.210060
           motor      0.086104  0.034200  0.176857
           semantic   0.080664  0.007559  0.164007
motor      glass      0.164101  0.000000  0.328202
           mem        0.156610  0.003536  0.312796
           semantic   0.142326  0.000000  0.284652
semantic   glass      0.097304  0.000000  0.194608
           mem        0.143203  0.003536  0.286752
           motor      0.120947  0.000000  0.241894

In [8]:
CV.groupby(['task']).mean()

,acc,spec,sen
task,,,
glass,0.878472,1.0,0.756944
mem,0.993750,1.0,0.987500
motor,0.981250,1.0,0.962500
semantic,0.943750,1.0,0.887500


In [9]:
CV.groupby(['task']).std()

,acc,spec,sen
task,,,
glass,0.064065,0.0,0.128131
mem,0.017678,0.0,0.035355
motor,0.025877,0.0,0.051755
semantic,0.032043,0.0,0.064087


In [11]:
DS.groupby(['task']).mean()

,acc,spec,sen
task,,,
glass,0.577480,0.996071,0.158889
mem,0.894375,0.821964,0.966786
motor,0.788393,0.843750,0.733036
semantic,0.669911,0.998214,0.341607


In [12]:
DS.groupby(['task']).std()

,acc,spec,sen
task,,,
glass,0.101508,0.014607,0.208554
mem,0.100476,0.210743,0.067017
motor,0.154729,0.282261,0.311407
semantic,0.144881,0.005430,0.291627


In [14]:
BS.groupby(['train_task','test_task']).mean()

acc      spec       sen
train_task test_task                              
glass      mem        0.571042  0.996071  0.146012
           motor      0.584782  0.996071  0.173492
           semantic   0.577837  0.996071  0.159603
mem        glass      0.764921  0.821726  0.708115
           motor      0.821071  0.821964  0.820179
           semantic   0.790804  0.821964  0.759643
motor      glass      0.605476  0.843770  0.367183
           mem        0.620179  0.843750  0.396607
           semantic   0.616518  0.843750  0.389286
semantic   glass      0.555040  0.998214  0.111865
           mem        0.562411  0.998214  0.126607
           motor      0.553036  0.998214  0.107857

In [16]:
BS.groupby(['train_task','test_task']).std()

acc      spec       sen
train_task test_task                              
glass      mem        0.114925  0.014607  0.235733
           motor      0.102336  0.014607  0.212573
           semantic   0.094114  0.014607  0.191819
mem        glass      0.131100  0.210683  0.243705
           motor      0.121392  0.210743  0.197581
           semantic   0.118036  0.210743  0.192986
motor      glass      0.098486  0.282237  0.338109
           mem        0.114638  0.282261  0.376767
           semantic   0.116396  0.282261  0.356059
semantic   glass      0.087032  0.005430  0.175022
           mem        0.095594  0.005430  0.192347
           motor      0.085810  0.005430  0.171620

In [ ]:
import pandas as pd
text=[181.0, 172.0]
df=pd.DataFrame(text)